On import les bibliotheques :

In [1]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON
import json

from pandas import json_normalize
import pandas as pd    
import json

import requests
import shutil
import os

import urllib.request
from PIL import Image
import numpy

On crée la fonction de création du JSON

In [7]:
####################################### CRÉATION JSON ################################################# a run

def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

def crea_json():
    endpoint_url = "https://query.wikidata.org/sparql"

    #query = """SELECT ?gateau ?gateauLabel ?gateauDescription ?image ?origineLabel
    #{
    #?gateau wdt:P279/wdt:P31* wd:Q477248; #des pâtisseries
    #        wdt:P18 ?image; #l'image
    #        wdt:P495 ?origine. # l'origine du gateau
                
    
    #SERVICE wikibase:label { bd:serviceParam wikibase:language "en"}

    #}
    #LIMIT 1000"""

    #query = """SELECT ?animal ?animalLabel ?animalDescription ?image 
    #{
    #?animal wdt:P279 wd:Q144; #des pâtisseries
    #        wdt:P18 ?image; #l'image
                
    
    #SERVICE wikibase:label { bd:serviceParam wikibase:language "en"}

    #}
    #LIMIT 1000"""

    query="""SELECT ?animal ?animalLabel ?animalDescription ?image 
    {
    ?animal wdt:P279/wdt:P31* wd:Q144; #des pâtisseries
            wdt:P18 ?image; #l'image
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en"}

    }
    LIMIT 100"""

    results = get_results(endpoint_url, query)
    dico_json ={}

    for result in results["results"]["bindings"]:
        #print(result)
        # on récupère le nom du gateau
        gateau = result['animalLabel']['value']
        # les valeurs qui nous interessent dans notre requête
        dico={}
        for key, val in result.items():
            dico[key]=val['value']
        dico_json[gateau]=dico
        #print("\n", gateau,'\n')
        #print(dico_json[gateau], '\n')

    #print(' dico json \n')
    #print(dico_json)
    with open('./fichier.json', 'w') as fichier :
        json.dump(dico_json, fichier)

crea_json()

On télécharge nos images

In [ ]:
############################################################################# a run
def get_imgs():
    # the name of the json with all the data
    json_path="fichier.json"
    # we get the json
    json_data = json.load(open(json_path))


    for key, info in json_data.items():
        #print('\n', key,'\n', info, '\n')
        name = info['animalLabel']# the name to save the image
        print(info['image'],"\n")
        urllib.request.urlretrieve(info['image'], "./images/"+name+".jpg")

get_imgs()

On cherche à récupérer les méta datas de chaque images

In [9]:
from PIL import Image
from PIL.ExifTags import TAGS 
import pandas as pd
import numpy
#from exif import Image

import os



def recup_matadata_img(path):
    """ Cette fonction récpère les métadatas d'une image si elle en a s et les stockes dans un data frame et le retourne
    INPUT :
        path : the path of the image to get the meta-data from
    OUTPUT :
        df : the f=data frame with all the meta-data we could get
    """
    #path =""# the image's path

    image = Image.open(path)# reading the image
   
    exifdata = image.getexif() #extracting the data
    # two list for the creation of the data frame
    keys=[]
    values=[]
    print(exifdata)
    lst_exif=['Copyright', '']

    for tag_id in exifdata: # we need the human redeable tags for the data, we get them with the function TAGS.get(id, id)
    
        tag = TAGS.get(tag_id, tag_id)# the human redeable tag
        data = exifdata.get(tag_id)# the value of the tag
        
        if isinstance(data, bytes): # decoding the tags data
            data = data.decode()
        keys.append(tag)
        print(tag,' ', data, '\n')
        values.append(str(data))
    
    ar=numpy.array([values])# the array for the data frame
    df = pd.DataFrame(ar, columns=keys)# the data frame
        
    return df

#################################### Some tests ###########################
print('result \n')
recup_matadata_img("images/Akita.jpg")
print(recup_matadata_img("images/Cane Corso.jpg"))
###########################################################################

def concat_lst_frame(lst):
    """ Cette fonction stocke la liste sous la forme d'un tuple"""
    print('Concaténation des frames \n')
    tup = ()
    for df in lst:
        tup =(tup, df)
    #print(tup)
    return tup



result 

{}
{}
Empty DataFrame
Columns: []
Index: [0]


On cherche à sauvegarder ces datas

In [11]:
import csv
from PIL import Image
import numpy

def recup_matadata_img(path):
    """ Cette fonction récpère les métadatas d'une image si elle en a s et les stockes dans un data frame et le retourne
    INPUT :
        path : the path of the image to get the meta-data from
    OUTPUT :
        df : the f=data frame with all the meta-data we could get
    """
    #path =""# the image's path

    image = Image.open(path)# reading the image
   
    exifdata = image.getexif() #extracting the data
    # two list for the creation of the data frame
    keys=[]
    values=[]
    #print(exifdata)

    for tag_id in exifdata: # we need the human redeable tags for the data, we get them with the function TAGS.get(id, id)
    
        tag = TAGS.get(tag_id, tag_id)# the human redeable tag
        data = exifdata.get(tag_id)# the value of the tag
        
        if isinstance(data, bytes): # decoding the tags data
            data = data.decode()
        keys.append(tag)
        values.append(str(data))
    
    ar=numpy.array([values])# the array for the data frame
    df = pd.DataFrame(ar, columns=keys)# the data frame
        
    return df

def save_meta_data(path):

    with open('meta_data.csv', 'w') as file:
        writer_obj=csv.writer(file, delimiter='|')

        dico = recup_matadata_img(path)#getting the meta_datas of the image
        header, values = [], []

        for key, value in dico.items():# the values to write into our csv file
                header.append(key)
                values.append(value)
                
            
        #print(values)
        writer_obj.writerow(header)# we save it in the csv file
        writer_obj.writerow(values)

############################################ Some tests #####################################
save_meta_data("images/Akita.jpg")
#############################################################################################

def stock_meta_data(path):
    
    files = os.listdir(path)
    for name in files :
        try :
            print(name)
            recup_matadata_img("images/"+name)
            print("done")
        except :
            print(name)
            print("error")

We get the meta datas ans stock it with data frames

In [ ]:
######################################################################################## a run
import os
import pandas as pd
from PIL import Image


def recup_matadata_img(path):
    """ Cette fonction récpère les métadatas d'une image si elle en a s et les stockes dans un data frame et le retourne
    INPUT :
        path : the path of the image to get the meta-data from
    OUTPUT :
        df : the f=data frame with all the meta-data we could get
    """
    #path =""# the image's path

    image = Image.open(path)# reading the image
   
    exifdata = image.getexif() #extracting the data
    # two list for the creation of the data frame
    keys=[]
    values=[]
    #print(exifdata)

    for tag_id in exifdata: # we need the human redeable tags for the data, we get them with the function TAGS.get(id, id)
    
        tag = TAGS.get(tag_id, tag_id)# the human redeable tag
        data = exifdata.get(tag_id)# the value of the tag
        
        if isinstance(data, bytes): # decoding the tags data
            data = data.decode()
        keys.append(tag)
        values.append(str(data))
    
    ar=numpy.array([values])# the array for the data frame
    df = pd.DataFrame(ar, columns=keys)# the data frame
    print(df)
        
    return df



def stock_meta_datas(path, csv_name):
    """ This function stock the meta-datas of the images of a whole folder
    INPUT:
        path : the path of the folder we get the images from
        csv_name : the name of the csv file whre we will stock the image's meta-data
    OUTPUT:
        NONE, the meta-datas are stored into a csv file
    """
    files = os.listdir(path)# we get all the photos from the folder
    lst = []
    counter=0# to check we do not an loose an existing frame
    for name in files :
        try :
            print(name)
            lst.append(recup_matadata_img("images/"+name))
            counter+=1
            #print("done")
        except :
            #print(name) 
            #print("error")
            pass
    frame=pd.concat(lst) # creating a big frame with all the images data-frame
    #print(frame)
    #print("On a {} frame au final ".format(len(lst)) )
    #print("\n normaleent on doit avoir {} frames".format(counter))
    frame.to_csv(csv_name, sep='|',encoding='utf-8', index=False, escapechar=" ")#writting into our csv file

#####################################################Some test ################################
stock_meta_datas('images/', "meta_data.csv")


Now we need to be able to find the stronger color ( the most present one)

In [ ]:
%pip install scikit-learn

In [ ]:
from PIL import Image
import numpy
import math
import matplotlib.pyplot as plot
from sklearn.cluster import KMeans



imgfile = Image.open("images/Alivenci.jpg")
numarray = numpy.array(imgfile.getdata(), numpy.uint8)
clusters = KMeans(n_clusters=4, n_init=2)
clusters.fit(numarray)
npbins = numpy.arange(0, 5)
histogram = numpy.histogram(clusters.labels_, bins=npbins)
labels = numpy.unique(clusters.labels_)
barlist = plot.bar(labels, histogram[0])
for i in range(4):
    barlist[i].set_color(
        "#%02x%02x%02x"
        % (
            math.ceil(clusters.cluster_centers_[i][0]),
            math.ceil(clusters.cluster_centers_[i][1]),
            math.ceil(clusters.cluster_centers_[i][2]),
        )

    )
    print(clusters.cluster_centers_[i][0], clusters.cluster_centers_[i][1], clusters.cluster_centers_[i][2], "\n")
plot.show()

Now we need to find the color, because we have the center of the clusters but it doesn't match a word but numarical values

In [13]:
import csv

def find_color(lst):
    colors=[]
    with open('all_colors.csv', 'r') as f:
        obj_read = csv.reader(f)

        for ligne in obj_read:
            #print(ligne[1].split('-'))
            values = ligne[1].split('-')

            # if the values in the lst input match the values of the line the color of the input is teh one of the line
            if (lst[0]==values[0]) and (lst[1]==values[1]) and (lst[2]==values[2]):
                print( 'La couleur est {}'.format(ligne[4]))
                colors.append(ligne[4])
                return(ligne[4])
 
        return str(lst)

####################################### Some tests ######################################################################

print(find_color(['100', '100', '100'])) 



['100', '100', '100']


We test it

In [15]:
######################################################################################## a run
from PIL import Image
import numpy
import math
import matplotlib.pyplot as plot
from sklearn.cluster import MiniBatchKMeans
import csv


def find_color(lst):
    """ this function find the name of the color input 
    INPUT :
    OUTPUT : 
    """
    colors=[]
    with open('all_colors.csv', 'r') as f:
        obj_read = csv.reader(f)

        for ligne in obj_read:
            #print(ligne[1].split('-'))
            values = ligne[1].split('-')

            # if the values in the lst input match the values of the line the color of the input is teh one of the line
            if (lst[0]==values[0]) and (lst[1]==values[1]) and (lst[2]==values[2]):
                print( 'La couleur est {}'.format(ligne[4]))
                colors.append(ligne[4])
                return(ligne[4])
 
        return str(lst)


def k_means(path):
    """ This function applies the k_means to the image (path)"""
    imgfile = Image.open(path) #"images/Alivenci.jpg"
    numarray = numpy.array(imgfile.getdata(), numpy.uint8)
    clusters = MiniBatchKMeans(n_clusters=4, n_init=2)
    clusters.fit(numarray)
    npbins = numpy.arange(0, 5)
    histogram = numpy.histogram(clusters.labels_, bins=npbins)
    labels = numpy.unique(clusters.labels_)
    
    colors=[]
    for i in range(4) :
        color=find_color( [ math.ceil(clusters.cluster_centers_[i][0]), math.ceil(clusters.cluster_centers_[i][1]), math.ceil(clusters.cluster_centers_[i][2]) ] )
        colors.append(color)
    return colors

##################################### Some test ###################################
#print(k_means("images/Alivenci.jpg"))


Now we need a file that memorise the colors and some parameters for each image

In [16]:
######################################################################################## a run
import os
import pandas as pd
import json
import numpy


def data_tree(image):
    """ This function applies the k_means to all the images in the data base and return a data frame with the dominant colors"""
    values=[]
    column =[]
    try :
        print(image)
        values.append(k_means(image)) # we get the main colors
        for i in range(1, len(values)): # we add the necessary columns
            column.append("color_{}".format(i))
        column.append("name")
        values.append(str(image))
    
    except :
        print("error")

    ar=numpy.array([values])# the array for the data frame
    df=pd.DataFrame(ar, columns=column)# the data frame
    
    return df

# we get the info of our images from the json
with open('fichier.json') as file:
    data=json.load(file)
#print(data)

lst_df=[]# a list to stock all the data frames of the image's tags


for key, dico in data.items():
    # the lists for the dataframe creation
    column=['name', 'label', 'description' ]
    values=[key, dico['animalLabel'],dico['animalDescription']]

    try :
        #print(key)
        colors = k_means('images/'+key+'.jpg')
        #print(colors)
        for i in range(1, len(colors)+1):# we add the necessary columns
            values.append(colors[i-1])
            column.append("color_{}".format(i))
            print('add color to list')
    
    except :
        print("error")
        pass


    # creation of the data frame
    #print(values)
    ar = numpy.array([values])# the array for the data frame
    lst_df.append(pd.DataFrame(ar, columns=column))# the data frame

tableau =pd.concat(lst_df)
print(tableau)
tableau.to_csv("img_tags.csv", sep=':', escapechar='|')

Chihuahua
['[132, 112, 85]', '[13, 22, 14]', '[217, 173, 133]', '[67, 74, 52]']
add color to list
add color to list
add color to list
add color to list
Great Dane
['[222, 211, 190]', '[59, 51, 37]', '[175, 152, 111]', '[218, 32, 5]']
add color to list
add color to list
add color to list
add color to list
American Bully
['[185, 154, 142]', '[47, 38, 58]', '[231, 212, 206]', '[108, 95, 107]']
add color to list
add color to list
add color to list
add color to list
Affenpinscher
['[116, 154, 94]', '[25, 28, 25]', '[164, 212, 128]', '[70, 88, 64]']
add color to list
add color to list
add color to list
add color to list
Austrian Black and Tan Hound
['[97, 96, 90]', '[191, 193, 199]', '[147, 147, 140]', '[48, 47, 49]']
add color to list
add color to list
add color to list
add color to list
Dalmatian
['[190, 169, 154]', '[31, 17, 25]', '[223, 213, 217]', '[123, 95, 92]']
add color to list
add color to list
add color to list
add color to list
Maremma Sheepdog
['[106, 116, 122]', '[194, 206, 219

In [4]:
import pandas as pd

sortie = pd.read_csv('img_tags.csv', sep=':')
#print(sortie)

In [8]:
from PIL import Image
import pandas as pd
import numpy
#print(sortie['name'])

def user_question():
    choix = int(input('Mettre cette image en favorite ? 1/0'))
    if choix == 1:
        return 'favorite'
    return 'NotFavorite'

def user_preferencies(dataFrame,column):
    values=[]
    counter = (len(column) -2)*10
    favorite=[]
    user_name= str(input('Enter your user name : '))
    column.append('Favorite')
    counter = 10

    for name in dataFrame['name'] :# for each picture
        if counter > 0:
            imageLue = Image.open('images/'+name+'.jpg')
            imageLue.show()
            # making a new row for the data frame
            row=list(dataFrame.iloc[10-counter]) #len(column)-2)*10
            row.append(user_question())
            values.append( row) # we save the tags of the picture
            counter -= 1
            print(counter)

        else :
            exit
    return user_name, values, column #pd.DataFrame( numpy.array(values), columns=column )

tags = pd.read_csv('img_tags.csv', sep=':')


truc = user_preferencies(tags, tags.columns.tolist())

9
8
7
6
5
4
3
2
1
0


In [10]:
test=pd.DataFrame( numpy.array(truc[1]), columns=truc[2] )
#print(test.values[1:1])
test.to_csv('user.csv')

we need to get the data from the user

In [62]:
user_data = pd.read_csv('user.csv')
print(len(user_data))
data=[]
result=[]

# we make a list of the tags the user already saw on a same image (data) and get the favorite or not (result)
for i in range(len(user_data)):
    data.append( user_data.loc[i][2:-1].to_list())
    result.append(user_data.loc[i][-1])
    #print(user_data.loc[i][2:-1].to_list() )
#print(data)
print(result)

column = user_data.columns.to_list()[2:]
print(column)
column = user_data.columns.to_list()[2:]

10
['favorite', 'NotFavorite', 'NotFavorite', 'favorite', 'favorite', 'NotFavorite', 'favorite', 'NotFavorite', 'NotFavorite', 'NotFavorite']
['name', 'label', 'description', 'color_1', 'color_2', 'color_3', 'color_4', 'favorite']


Now we need 

Now we need to draw the classifier 

In [45]:
from sklearn import tree
import pandas as pd
from sklearn.preprocessing import LabelEncoder


# creating dataframes
column = user_data.columns.to_list()[2:]
dataframe = pd.DataFrame(data, columns=column[:-1])
resultframe = pd.DataFrame(result, columns=[column[-1]])

# the list of the encoded labels
list_label=[]

#Encoding the labels
for i in range( len(column)-1 ):
    #print('label en cours ', column[i])
    label = column[i]# the name of the label to encode
    list_label.append(LabelEncoder())# a label object
    label_encoded = list_label[i].fit_transform(dataframe[label]) # the encoded label
    dataframe[label]= label_encoded

# the label for the results
list_label.append(LabelEncoder())
resultframe[column[-1] ] = list_label[-1].fit_transform( resultframe[column[-1]])

# Use of decision tree classifiers
dtc = tree.DecisionTreeClassifier()
dtc = dtc.fit(dataframe.values, resultframe)


We try to predict now

In [74]:
# on vérifit les prédictions pour tt les images que l'utilisateur a déjà vues
for i in range (len(user_data)):

    # the tags of the image
    tags = user_data.loc[i][2:-1]
    len_tags=len(tags)
    counter = len_tags
    #the list of the tags we want to predict the result
    lst=[]
    
    # on test les tags de chaques images (ils appartiennet a des labels)
    for a in range(len(list_label)-1):
        label_object= list_label[a]
        lst.append(label_object.transform([tags[len_tags-counter]])[0] )
        counter -=1

    # we predict the result favorite or not depending on the tags
    prediction = dtc.predict( [lst] )
    print('Résultat : ',list_label[-1].inverse_transform(prediction), ' Résultat attendu : ', user_data.loc[i][-1])
    print(tags.to_list())
    print(dtc.feature_importances_, '\n')


Résultat :  ['favorite']  Résultat attendu :  favorite
['Chihuahua', 'Chihuahua', 'dog breed', '[132, 112, 85]', '[13, 22, 14]', '[217, 173, 133]', '[67, 74, 52]']
[0.         0.26785714 0.         0.35714286 0.         0.
 0.375     ] 

Résultat :  ['NotFavorite']  Résultat attendu :  NotFavorite
['Great Dane', 'Great Dane', 'dog breed', '[222, 211, 190]', '[59, 51, 37]', '[175, 152, 111]', '[218, 32, 5]']
[0.         0.26785714 0.         0.35714286 0.         0.
 0.375     ] 

Résultat :  ['NotFavorite']  Résultat attendu :  NotFavorite
['American Bully', 'American Bully', 'dog breed', '[185, 154, 142]', '[47, 38, 58]', '[231, 212, 206]', '[108, 95, 107]']
[0.         0.26785714 0.         0.35714286 0.         0.
 0.375     ] 

Résultat :  ['favorite']  Résultat attendu :  favorite
['Affenpinscher', 'Affenpinscher', 'dog breed', '[116, 154, 94]', '[25, 28, 25]', '[164, 212, 128]', '[70, 88, 64]']
[0.         0.26785714 0.         0.35714286 0.         0.
 0.375     ] 

Résultat :  